This is a tutorial pyflowline notebook.
This tutorial is an example of the pyflowline application using a Model for Prediction Across Scales (MPAS) mesh.

The following publication includes a comprehensive application:
Liao, C., Zhou, T., Xu, D., Cooper, M. G., Engwirda, D., Li, H.-Y., & Leung, L. R. (2023). Topological relationship-based flow direction modeling: Mesh-independent river networks representation. Journal of Advances in Modeling Earth Systems, 15, e2022MS003089. https://doi.org/10.1029/2022MS003089

The full documentation is hosted at: https://pyflowline.readthedocs.io

In order the run this notebook, you must install the PyFlowline package and its dependencies. 
Besides, the visulization requires the optional dependency packages (see the full documentation installation section).
You can also modify the notebook to use a different visualization method.


First, let's import some Python libraries.

In [ ]:
import os
import json
from pathlib import Path
from os.path import realpath
from datetime import datetime
import importlib.util

Next, we'll ensure the dependencies are satisfied before proceeding.

In [ ]:
required_packages = {
    "gdal": "osgeo",
    "numpy": "numpy",
    "netCDF4": "netCDF4",
    "pyearth": "pyearth",
    "pyflowline": "pyflowline",
    "geopandas": "geopandas",
    "matplotlib": "matplotlib",
    "requests": "requests",  # This one is only for downloading the data
}
missing_packages = [pkg for pkg, mod in required_packages.items() if not importlib.util.find_spec(mod)]

if missing_packages:
    print("The following packages are not installed:", ", ".join(missing_packages))
    print("If any dependency is missing, please install it using conda.")
else:
    print("Congrats, all dependencies are installed.")

The first step to running a pyflowline simulation is to configure the simulation. 

Configuration is separated into two levels: a "parent" configuration file which defines the domain, and a "child" configuration file which defines individual basins within the domain. The latter is referred to as the basin configuration file.

![pyflowline model structure](../../docs/figures/structure_pyflowline.png)

The pyflowline package uses json configuration files. Template configuration files are provided in the `data/` folder of this repo.

To configure a new case, pyflowline provides functions to read the configuration files, and programatically change the configuration parameters (json key values).

Import the pyflowline package functions we will use in this tutorial. 

In [ ]:
from pyflowline.configuration.read_configuration_file import pyflowline_read_configuration_file
from pyflowline.configuration.change_json_key_value import change_json_key_value

To define a new PyFlowline case, set the domain name and the path parameters.

In [ ]:
# Set the domain name and the "parent" configuration file name.
sRegion = "susquehanna"
sFilename_domain_config_in = 'pyflowline_susquehanna_mpas.json'
sFilename_basins_config_in = 'pyflowline_susquehanna_basins.json'

# Set paths to where the input data exists, and where the outputs are written.
sPath_project = str(Path().resolve().parents[1]) 
sPath_input = os.path.join(sPath_project, 'data', sRegion, 'input')
sPath_output = os.path.join(sPath_project, 'data', sRegion, 'output')

# Set the full path to the domain (parent) basins (child) configuration files.
sFilename_domain_config_in = realpath(os.path.join(sPath_input, sFilename_domain_config_in))
sFilename_basins_config_in = realpath(os.path.join(sPath_input, sFilename_basins_config_in))

# Print paths to the screen to confirm they are correct.
print("sPath_project: " + sPath_project)
print("sPath_input: " + sPath_input)
print("sPath_output: " + sPath_output)
print("sFilename_domain_config_in: " + sFilename_domain_config_in)
print("sFilename_basins_config_in: " + sFilename_basins_config_in)

For this notebook, we will download an unstructured mesh file from GitHub, created with the Model for Prediction Across Scales. 

Set the remote filename and the local filename where the MPAS mesh will be saved.

In [ ]:
sFilename_mpas_remote = 'https://github.com/changliao1025/pyflowline/releases/download/0.2.0/lnd_cull_mesh.nc'
sFilename_mpas_local = realpath(os.path.join(sPath_input, 'mpas_mesh.nc'))

Download the MPAS mesh NetCDF file stored in the following Github release:
https://github.com/changliao1025/pyflowline/releases/tag/0.2.0
https://github.com/changliao1025/pyflowline/releases/download/0.2.0/lnd_cull_mesh.nc

In [ ]:
import requests

# Create the data/<domain>/input directory if it does not exist
os.makedirs(sPath_input, exist_ok=True)

# Check whether the file exists
if os.path.exists(sFilename_mpas_local):
    print("File exists. Skipping download.\n" + sFilename_mpas_local)
else:
    print("Downloading to:\n" + sFilename_mpas_local)
    # Send an HTTP GET request to the URL
    response = requests.get(sFilename_mpas_remote)

    # Check if the request was successful
    if response.status_code == 200:
        # Save the content of the response to the local file
        with open(sFilename_mpas_local, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded successfully.")
    else:
        print(f"Failed to download file from:\n'{sFilename_mpas_remote}'")

pyflowline uses a json file for configuration, an example json file is provided.

Check the contents of the json configuration file.

In [ ]:
with open(sFilename_domain_config_in, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed, indent=4))

The meaning of these json keywords are explained in the [pyflowline documentation](https://pyflowline.readthedocs.io/en/latest/data/data.html#inputs).

For some parameters, we can change them using a function call, demonstrated below. For some other parameters (e.g., file paths), you need to modify the json file using a text editor. If the function returns an error, you should update the json file(s). 

Now set up some keywords which define the parameters for this case.

In [ ]:
#set up some parameters
sMesh_type = 'mpas'
iCase_index = 1
dResolution_meter = 5000 # mesh resolution
sDate = datetime.now().strftime('%Y%m%d')

We need to update a few parameters in the configuration file before we can create the flowline object.

The json file will be overwritten, you may want to make a copy of it first.

In [ ]:
# The boundary to clip the mesh
sFilename_mesh_boundary = realpath(os.path.join(
    sPath_input, 'boundary_wgs.geojson'))

# Set the path to the mpas file we just downloaded
change_json_key_value(sFilename_domain_config_in, 'sFilename_mesh_netcdf', sFilename_mpas_local) 

# Set the path to the boundary file used to clip the mesh
change_json_key_value(sFilename_domain_config_in,  'sFilename_mesh_boundary', sFilename_mesh_boundary) 

# Set the path to the individual-basin ("child") configuration file
change_json_key_value(sFilename_domain_config_in, 'sFilename_basins', sFilename_basins_config_in) 

# Set the path to the output folder
change_json_key_value(sFilename_domain_config_in, 'sWorkspace_output', sPath_output)

# Now change basin configuration file
sFilename_flowline = realpath(os.path.join(sPath_input, 'flowline.geojson') )

# Set the path to the user-provided flowline. Note that when changing the basin ("child") configuration file, set iFlag_basin_in=1.
change_json_key_value(sFilename_basins_config_in,  'sFilename_flowline_filter', sFilename_flowline, iFlag_basin_in=1)

The pyflowline package uses the OOP approach to manage each simulation. A pyflowline object is created by reading the configuration file.

The first argument to the `pyflowline_read_configuration_file` function is the configuration file filename, followed by name-value keywords that correspond to parameters in the json configuration files.

Some useful keyword arguments are:
- `iCase_index_in`: this is an arbitrary ID to identify the simulation case.
- `sMesh_type_in`: this specifies the mesh type ('mpas' in this example).
- `sDate_in`: this specifies the date of the simulation, the final output folder will have a pattern such as 'pyflowline20230901001', where pyflowline is model, 20230901 is the date, and 001 is the case index.

Call the function to create a pyflowline object.

In [ ]:
oPyflowline = pyflowline_read_configuration_file(sFilename_domain_config_in, iCase_index_in=iCase_index, sMesh_type_in=sMesh_type, sDate_in=sDate)

In addition to the `pyflowline_read_configuration_file` function, users can change model parameters after creating the model object.

In this example, we will change the mesh file name.

In [ ]:
oPyflowline.pyflowline_change_model_parameter('sFilename_mesh_netcdf', sFilename_mpas_local)

In general, a digital elevation model (DEM) is not required by `pyflowline`, but is required by `hexwatershed`. If the model configuration file includes the parameter `sFilename_dem`, `pyflowline` will use the DEM file to define the boundary and spatial reference of the domain. In this case, a domain boundary file is not required.

Alternatively, the user can supply a domain boundary file by setting the `sFilename_mesh_boundary` parameter. This file becomes required if `sFilename_dem` is not provided. 

The MPAS mesh is unique in that it contains elevation data associated with each mesh cell, therefore we don't have a separate DEM file, and instead we need to supply a domain boundary and set the `sFilename_mesh_boundary` parameter.

In [ ]:
oPyflowline.pyflowline_change_model_parameter('sFilename_mesh_boundary', sFilename_mesh_boundary)

We can also set parameters for individual basins in the domain. In this example, we only have one basin.

Although each basin can have different parameters, `pyflowline_change_model_parameter` does not currently support changing parameters for individual basins (it sets the same parameter value for all of the basins). If you want to set different parameter values for individual basins (for example, basin 1 has no dam, but basin 2 has a dam), you should edit the basin configuration json file directly. 

When setting a basin configuration file parameter, we must set the final argument `iFlag_basin_in=1`.

In [ ]:
oPyflowline.pyflowline_change_model_parameter('iFlag_dam', 0, iFlag_basin_in=1)

You can review the setting again.

In [ ]:
print(oPyflowline.tojson())

Another important (and required) parameter is the approximate outlet location, which `pyflowline` uses as a starting point for its upstream walk. In a typical workflow, we suggest you plot your flowline in software such as QGIS, visually identify the coordinates, and either type them directly into the model configuration json file, or update them programmatically as shown below.

Set the approximate outlet location coordinates using the `pyflowline_change_model_parameter` function.

In [ ]:
oPyflowline.pyflowline_change_model_parameter(
    'dLongitude_outlet_degree', -76.0093, 
    iFlag_basin_in = 1)
oPyflowline.pyflowline_change_model_parameter(
    'dLatitude_outlet_degree', 39.4620, 
    iFlag_basin_in = 1)

You can check the settings for individual basins as well. Here there is a single basin:

In [ ]:
print(oPyflowline.aBasin[0].tojson())

After the case object was created, we can set up the model.

In [ ]:
oPyflowline.pyflowline_setup()

Before any operation, we can visualize the original or raw flowline dataset. 

In [ ]:
if "geopandas" in missing_packages:
    print("Geopandas is not installed.")
else:
    import geopandas as gpd
    import matplotlib.pyplot as plt
    # Use the geopandas package
    # The raw/original geojson file 
    sFilename_geojson = oPyflowline.aBasin[0].sFilename_flowline_filter_geojson
    gdf = gpd.read_file(sFilename_geojson)
    gdf.plot()
    plt.show()

You can also use QGIS.

The plot function provides a few optional arguments such as map projection and spatial extent. 
By default, the spatial extent is full. 
But you can set the extent to a zoom-in region.

Now let's run the three major steps/operations in the pyflowline algorithm one by one.

In [ ]:
# Run step 1: flowline simplification.
oPyflowline.pyflowline_flowline_simplification();

Check the result using a plot.

In [ ]:
sFilename_geojson = oPyflowline.aBasin[0].sFilename_flowline_simplified
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()
pass

We can zoom in using the extent.

Next, create a mesh for the domain from the global MPAS mesh.

In [ ]:
# Run step 2: create a mesh.
oPyflowline.iFlag_mesh_boundary = 0 #set to 0 to disable polygon-based
oPyflowline.dLongitude_left= -79
oPyflowline.dLongitude_right= -74.5
oPyflowline.dLatitude_bot= 39.20
oPyflowline.dLatitude_top= 42.8

aCell = oPyflowline.pyflowline_mesh_generation()

Visualize the mesh.

In [ ]:
sFilename_geojson = oPyflowline.sFilename_mesh
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()
pass

We can also use a polygon to create a mesh.

In [ ]:
oPyflowline.iFlag_mesh_boundary = 1 #set to 1 to enable polygon-based
aCell = oPyflowline.pyflowline_mesh_generation()

In [ ]:
sFilename_geojson = oPyflowline.sFilename_mesh
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()

Last, we can generate the conceptual flowline. We refer to the final flowline as "conceptual" because it has been modified relative to the original, input flowline, which often represents a "real" flowline. The conceptual flowline has been simplified (e.g., small reaches, loops, and braided channels removed) and adjusted to align with the mesh. These modifications ensure the final flowline is suitable for hydrological modeling, while remaining consistent with the real flowline.

In [ ]:
# Step 3: create the "conceptual" (topological) flowline.
oPyflowline.pyflowline_reconstruct_topological_relationship();

Now we can overlap mesh with flowline.

In [ ]:
# Plot both the mesh and the flowline
file1_path = oPyflowline.sFilename_mesh
file2_path = oPyflowline.aBasin[0].sFilename_flowline_conceptual
gdf1 = gpd.read_file(file1_path)
gdf2 = gpd.read_file(file2_path)
fig, ax = plt.subplots()
gdf1.plot(ax=ax, color='blue')
gdf2.plot(ax=ax, color='red')
plt.show()
pass

After this, we can save the model output into a json file.

In [ ]:
# Export output to json.
oPyflowline.pyflowline_export();

The content of the one of the exported json files can be checked:

In [ ]:
with open(oPyflowline.sFilename_mesh_info, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed[0], indent=4))

In [ ]:
sFilename_flowline_conceptual_info = os.path.join(str(Path(
    oPyflowline.aBasin[0].sWorkspace_output_basin)), 
    oPyflowline.aBasin[0].sFilename_flowline_conceptual_info)

with open(sFilename_flowline_conceptual_info, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed[0], indent=4))

Congratulations! You have successfully finished a pyflowline simulation.